In [1]:
import os
import pandas as pd
import numpy as np
import openai
import chromadb
from tqdm import tqdm
from dotenv import load_dotenv

In [2]:
# Cargar variables de entorno desde .env
load_dotenv()

True

In [3]:
# Configuración
DATA_FILE = "../data/02_processed/clean_texts.csv"
EMBEDDINGS_DIR = "../data/03_embeddings/"
DB_NAME = "chroma_db"

In [4]:
# Cargar los datos
print("Cargando datos preprocesados...")
df = pd.read_csv(DATA_FILE)

Cargando datos preprocesados...


In [5]:
# Inicializar ChromaDB
chroma_client = chromadb.PersistentClient(path=EMBEDDINGS_DIR)
collection = chroma_client.get_or_create_collection(DB_NAME)


In [6]:
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [7]:

# Función para obtener embeddings
def get_embedding(text):
    """Obtiene el embedding de un texto usando OpenAI."""
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002" #text-embedding-ada-002
    )
    return response.data[0].embedding

In [8]:
# Generar embeddings y almacenarlos en ChromaDB
print("Generando embeddings y almacenando en ChromaDB...")
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row["clean_text"]
    embedding = get_embedding(text)
    collection.add(
        ids=[str(i)],
        embeddings=[embedding],
        documents=[row["clean_text"]], 
        metadatas=[{"filename": row["filename"], "chunk_id": row["chunk_id"]}]
    )

print(f"Base de datos vectorial guardada en {EMBEDDINGS_DIR}")

Generando embeddings y almacenando en ChromaDB...


100%|██████████| 68/68 [00:24<00:00,  2.83it/s]

Base de datos vectorial guardada en ../data/03_embeddings/
